In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('./classification220926/train.csv')
dftest = pd.read_csv('./classification220926/test.csv')

x = df.iloc[:,:-1]
x1_test = dftest.iloc[:, :-1]
y = df.loc[:,['income']]
# y1_test = dftest.loc[:,['income']]

In [113]:
x.drop(['no', 'education', 'fnlwgt', 'native-country'], axis=1, inplace=True)
x1_test.drop(['no', 'education', 'fnlwgt'],
             axis=1, inplace=True)
x

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week
0,25,Private,7,Divorced,Machine-op-inspct,Not-in-family,White,Male,0,0,40
1,39,Private,10,Divorced,Other-service,Not-in-family,White,Female,0,1721,55
2,35,Private,13,Never-married,Handlers-cleaners,Not-in-family,Asian-Pac-Islander,Female,0,0,50
3,64,Private,11,Divorced,Tech-support,Not-in-family,White,Female,0,0,40
4,24,Private,10,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,70
...,...,...,...,...,...,...,...,...,...,...,...
29300,20,Private,9,Never-married,Sales,Own-child,Other,Male,0,0,40
29301,21,Private,9,Never-married,Sales,Own-child,Black,Male,0,0,40
29302,18,Self-emp-not-inc,6,Never-married,Farming-fishing,Own-child,White,Male,0,0,40
29303,33,Private,7,Never-married,Craft-repair,Not-in-family,White,Male,0,0,65


In [114]:
x['workclass'] = x['workclass'].astype('category')
x['marital-status'] = x['marital-status'].astype('category')
x['relationship'] = x['relationship'].astype('category')
x['race'] = x['race'].astype('category')
x['sex'] = x['sex'].astype('category')
x['workclass'] = x['workclass'].astype('category')
x1_test['workclass'] = x1_test['workclass'].astype('category')
x1_test['marital-status'] = x1_test['marital-status'].astype('category')
x1_test['relationship'] = x1_test['relationship'].astype('category')
x1_test['race'] = x1_test['race'].astype('category')
x1_test['sex'] = x1_test['sex'].astype('category')
x1_test['workclass'] = x1_test['workclass'].astype('category')


In [115]:
x = pd.get_dummies(x)
x1_test = pd.get_dummies(x1_test)



In [116]:
from sklearn.model_selection import train_test_split #트레인과 test 비율 나눔
from sklearn.model_selection import cross_val_score  # k-fold cross validation
from sklearn.preprocessing import MinMaxScaler



x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=42)

scaler = MinMaxScaler()

x_train[['capital-gain']] = scaler.fit_transform(x_train[['capital-gain']])
x_test[['capital-gain']] = scaler.fit_transform(x_test[['capital-gain']])
x1_test[['capital-gain']] = scaler.fit_transform(x1_test[['capital-gain']])

x_train[['capital-loss']] = scaler.fit_transform(x_train[['capital-loss']])
x_test[['capital-loss']] = scaler.fit_transform(x_test[['capital-loss']])
x1_test[['capital-loss']] = scaler.fit_transform(x1_test[['capital-loss']])


In [130]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt = DecisionTreeClassifier(random_state=42, criterion='entropy')
dt.fit(x_train, y_train)  # 여기서 x_train은 이미 위에서 스케일링함

print(dt.score(x_train, y_train))  # 훈련 데이터
print(dt.score(x_test, y_test))  # 테스트 데이터



0.9795251791546824
0.7979981801637852


0.7979981801637852

In [118]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold

cv = KFold(n_splits=5, random_state=42, shuffle=True)
estimator = DecisionTreeClassifier()
params = {'max_depth': [10,12,14], 'min_samples_split': [10,12,14,16,18,20], 'random_state' : [24], 
          'criterion': ['entropy', 'gini'],'min_weight_fraction_leaf': [0.0],
          'min_impurity_decrease': [0.0]}


gridtree = GridSearchCV(estimator=estimator, param_grid=params,
                        cv=cv, verbose=-1, n_jobs=-1, refit=True)

gridtree.fit(x_train, y_train)

print("Best Estimator:\n", gridtree.best_estimator_)
print()
print("Best Params:\n", gridtree.best_params_)
print()
print("Best Score:\n", gridtree.best_score_)
print()



Best Estimator:
 DecisionTreeClassifier(criterion='entropy', max_depth=12, min_samples_split=18,
                       random_state=24)

Best Params:
 {'criterion': 'entropy', 'max_depth': 12, 'min_impurity_decrease': 0.0, 'min_samples_split': 18, 'min_weight_fraction_leaf': 0.0, 'random_state': 24}

Best Score:
 0.8521910274117724



In [119]:

print(gridtree.score(x_train, y_train))
print(gridtree.score(x_test, y_test))


0.8721298688636474
0.8322338489535942


In [121]:
dfsubmit = pd.read_csv('./classification220926/sample_submission.csv')

In [129]:
dfsubmit['income'] = gridtree.predict(x1_test)
dfsubmit.to_csv('model04_CLA.csv',index=False)
